In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from math import *
from scipy.special import logsumexp

In [ ]:
img = cv2.imread('messi.jpg', 0)
m, n = img.shape
thresholdMagnetude = np.zeros([m, n])
dx_img = np.zeros([m, n])
dy_img = np.zeros([m, n])
magnetude= np.zeros([m, n])
phase= np.zeros([m, n])

def sobal():
    sum=0
    dx= np.array([-1,0,1,-2,0,2,-1,0,1],dtype = int).reshape(3,3)
    dy= np.array([1,2,1,0,0,0,-1,-2,-1],dtype = int).reshape(3,3)
    temp = 0
    for i in range(1, m-1):
        for j in range(1, n-1):
            for k in range(3):
                for s in range(3):
                    dx_img[i, j]+= img[i-1 + k, j-1+s]*dx[k, s]

    for i in range(1, m-1):
        for j in range(1, n-1):
            for k in range(3):
                for s in range(3):
                    dy_img[i, j]+= img[i-1 + k, j-1+s]*dy[k, s]
                    
    for i in range(0,m):
        for j in range (0,n):
            magnetude[i,j]=np.sqrt((dx_img[i, j]*dx_img[i, j])+(dy_img[i, j]*dy_img[i, j]))
            temp = np.logaddexp(dy_img[i, j],dx_img[i, j])
            phase[i,j]= round(atan(temp) * 180 / pi )
            sum+= magnetude[i,j]

    threshold=sum/(m*n)
    for i in range(0,m):
        for j in range (0,n):
            if(magnetude[i,j]>threshold):
                thresholdMagnetude[i,j]=1
            else:
                thresholdMagnetude[i,j]=0
    
    imgplot= plt.imshow(img,cmap="gray")
    plt.title("Original Image")
    plt.show()
    imgplot= plt.imshow(dy_img,cmap="gray")
    plt.title("sobal_dy_horizontal")
    plt.show()
    
    plt.imshow(dx_img,cmap="gray")
    plt.title("sobal_dx_Vertical")
    plt.show()
    
    plt.imshow(magnetude,cmap="gray")
    plt.title("Magnetude")
    plt.show()
    
    plt.imshow(thresholdMagnetude,cmap="gray")
    plt.title("Threshold Magnetude")
    plt.show()
    

def enhance_thMagnitude(ith,jth,M,P):
    for i in range(ith-1,ith+2):
        for j in range(jth-1,jth+2):
            if thresholdMagnetude[i][j]==0:
                if np.abs(magnetude[ith][jth] - magnetude[i][j]) <= M and np.abs(phase[ith][jth] - phase[i][j]) <= P:
                    thresholdMagnetude[i][j] = 1

def edge_linking():
    #get magnitude threshold value by difference and 20%
    maxM = np.amax(magnetude)
    minM = np.amin(magnetude)
    magnetude_threshold = (maxM - minM)*3/100

    #get phase threshold value by difference and 20%
    maxP = np.amax(phase)
    minP = np.amin(phase)
    phase_threshold = (maxP - minP)*5/100
    
    for i in range(1,m-1):
        for j in range(1,n-1):
            if thresholdMagnetude[i][j]==1:
                enhance_thMagnitude(i,j,magnetude_threshold,phase_threshold)
    
    plt.imshow(thresholdMagnetude,cmap="gray")
    plt.title("Edge Linking")
    plt.show()


sobal()
edge_linking()

img = cv2.imread('messi.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edge = cv2.Canny(gray,50,150,apertureSize = 3)
cdst = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)
cdstP = np.copy(cdst)

lines = cv2.HoughLines(edge, 1, np.pi / 180, 150, None, 0, 0)
    
if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = cos(theta)
        b = sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        cv2.line(cdst, pt1, pt2, (0,0,255), 3, cv2.LINE_AA)
        
plt.imshow(cdst,cmap="gray")
plt.title("Standard Hough Line Transform")
plt.show()